In [4]:
import os
import numpy as np
import pandas as pd
import dotenv
import warnings
import mne 


dotenv.load_dotenv()
warnings.filterwarnings("ignore")

In [8]:
# there are 2 types of files in the directory; ended with MDD and HC -> count them
eegDirPath = "/Users/wachiii/Workschii/brain-mdd/data/16channels"

mddFiles = [f for f in os.listdir(eegDirPath) if f.endswith("MDD.npy")]
hcFiles = [f for f in os.listdir(eegDirPath) if f.endswith("HC.npy")]
# append in list
mddFiles = [os.path.join(eegDirPath, f) for f in mddFiles]
hcFiles = [os.path.join(eegDirPath, f) for f in hcFiles]
# sort them
mddFiles.sort()
hcFiles.sort()

# use first 24 files
mddFiles = mddFiles[:24]
hcFiles = hcFiles[:24]
print(f"Number of MDD files: {len(mddFiles)}")
print(f"Number of HC files: {len(hcFiles)}")

Number of MDD files: 24
Number of HC files: 24


In [19]:
import numpy as np
import mne

# Define paths
balancedDataPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels"

# Define constants
fs = 250  # Sampling frequency (Hz)
start_time = 3  # Start after 5 seconds
duration = 6 * 60  # 6 minutes in seconds
num_samples = duration * fs  # Expected total samples per subject

# Initialize lists to store data
mdd_data_all = []
hc_data_all = []

# Track the minimum length encountered
min_length = float("inf")

# Process each subject
for i, (mddFile, hcFile) in enumerate(zip(mddFiles[:24], hcFiles[:24])):  
    mddData = np.load(mddFile)
    hcData = np.load(hcFile)

    # Extract the desired time segment
    mddSegment = mddData[:, start_time * fs : start_time * fs + num_samples]
    hcSegment = hcData[:, start_time * fs : start_time * fs + num_samples]

    # Ensure segments have enough samples before adding
    min_length = min(min_length, mddSegment.shape[1], hcSegment.shape[1])

    mdd_data_all.append(mddSegment)
    hc_data_all.append(hcSegment)

    print(f"Processed {i+1}/24 subjects")

# Trim all data to the same minimum length
mdd_data_all = [m[:, :min_length] for m in mdd_data_all]
hc_data_all = [h[:, :min_length] for h in hc_data_all]

# Concatenate along time axis
mdd_data_all = np.hstack(mdd_data_all)
hc_data_all = np.hstack(hc_data_all)

# Create MNE info object
ch_names = [f"ch{i}" for i in range(16)]
ch_types = ["eeg"] * 16
info = mne.create_info(ch_names=ch_names, sfreq=fs, ch_types=ch_types)

# Create MNE RawArray object
mddRaw = mne.io.RawArray(mdd_data_all, info)
hcRaw = mne.io.RawArray(hc_data_all, info)

# Save to file
mddRaw.save(f"{balancedDataPath}/mdd1624.fif", overwrite=True)
hcRaw.save(f"{balancedDataPath}/hc1624.fif", overwrite=True)

print(f"Final shape: MDD {mddRaw.times[-1]}s, HC {hcRaw.times[-1]}s")
print("Saved mdd1624.fif and hc1624.fif successfully!")


Processed 1/24 subjects
Processed 2/24 subjects
Processed 3/24 subjects
Processed 4/24 subjects
Processed 5/24 subjects
Processed 6/24 subjects
Processed 7/24 subjects
Processed 8/24 subjects
Processed 9/24 subjects
Processed 10/24 subjects
Processed 11/24 subjects
Processed 12/24 subjects
Processed 13/24 subjects
Processed 14/24 subjects
Processed 15/24 subjects
Processed 16/24 subjects
Processed 17/24 subjects
Processed 18/24 subjects
Processed 19/24 subjects
Processed 20/24 subjects
Processed 21/24 subjects
Processed 22/24 subjects
Processed 23/24 subjects
Processed 24/24 subjects
Creating RawArray with float64 data, n_channels=16, n_times=1784136
    Range : 0 ... 1784135 =      0.000 ...  7136.540 secs
Ready.
Creating RawArray with float64 data, n_channels=16, n_times=1784136
    Range : 0 ... 1784135 =      0.000 ...  7136.540 secs
Ready.
Overwriting existing file.
Writing /Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif
Closing /Users/wachiii/Workschii/brai

In [20]:
fifFileMddPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif"
fifFileHcPath = "/Users/wachiii/Workschii/brain-mdd/data/balanced16channels/hc1624.fif"

# inspect the data
mddRaw = mne.io.read_raw_fif(fifFileMddPath)
hcRaw = mne.io.read_raw_fif(fifFileHcPath)

print(mddRaw)
print(hcRaw)

Opening raw data file /Users/wachiii/Workschii/brain-mdd/data/balanced16channels/mdd1624.fif...
Isotrak not found
    Range : 0 ... 1784135 =      0.000 ...  7136.540 secs
Ready.
Opening raw data file /Users/wachiii/Workschii/brain-mdd/data/balanced16channels/hc1624.fif...
Isotrak not found
    Range : 0 ... 1784135 =      0.000 ...  7136.540 secs
Ready.
<Raw | mdd1624.fif, 16 x 1784136 (7136.5 s), ~18 kB, data not loaded>
<Raw | hc1624.fif, 16 x 1784136 (7136.5 s), ~18 kB, data not loaded>


In [22]:
1784136/24

74339.0

In [21]:
7136.5/(24*60)

4.955902777777778